In [41]:
import sklearn
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from scipy import sparse

data = pd.read_csv('weights.csv')
WT = pd.read_csv('wisdomtree.csv')
WT = list(WT['Ticker'])
#df to pivot table
#matrix = df.pivot_table(index = 'user_id', columns ='item_id', values ='rating')
matrix = data.pivot_table(index = 'Account__c', columns ='Ticker__c', values ='AUM_weight')
data = matrix.fillna(0)
#drop columns that are all zero
data = data.loc[:, (data != 0).any(axis=0)]
data.insert(0, 'user', range(0, 0 + len(data)))
data_items = data.drop('user', axis =1)
#data.to_csv(r'documents/2019-04/New Fund/data_matrix2.csv')


"""ITEM TO ITEM CALCULATION"""
# normalize user vectors to unit vectors
# magnitude = sqrt(x2 + y2 + z2 + ...)
#magnitude = np.sqrt(np.square(data_items).sum(axis=1))
# unit vector = (x / magnitude, y / magnitude, z/ magnitude ...)
#data_items = data_items.divide(magnitude, axis = 'index')
#data_items.to_csv(r'documents/2019-04/New Fund/data_items2.csv')

"""build function: calculate the column-wise cosine similarity and store into new df"""
def calculate_similarity(data_items):
    #"store matrix into sparse format"
    data_sparse = sparse.csr_matrix(data_items)
    """transpose the df since cos sim function only works for rows and find similarities"""
    similarities = cosine_similarity(data_sparse.transpose()) 
    """store into dataframe"""
    sim = pd.DataFrame(data = similarities, index= data_items.columns, columns= data_items.columns)
    return sim

"""cosine similairty matrix"""
data_matrix2 = calculate_similarity(data_items)

"""only keep wisdomtree columns in matrix"""
wisdomtree_list = []
for col in list(data_matrix2.columns.values):
    if col in WT:
        wisdomtree_list.append(col)
data_matrix = data_matrix2[wisdomtree_list]

"""USER TO ITEM CALCULATION
new dataframe with k closest neighbours (most similar) for each ticker"""

"""create columns for top 6 recommendations"""
data_neighbours = pd.DataFrame(index=data_matrix.index, columns=range(1,7))

for i in range(0, len(data_matrix.index)):
    """"the neighbours = selecting the column and choosing the top 5"""
    """i is the column number for the ticker"""
    #note: column 1, 2, 3 in csv = index 0, 1, 2
    data_neighbours.iloc[i,:6] = data_matrix.iloc[i, 0:].sort_values(ascending=False)[:6].index

#make new blank column in original data frame for each user's recommendation 
data['bought tickers'] = ""
data['recommendation'] = ""
#data_matrix.to_csv(r'documents/2019-04/New Fund/data_matrix2.csv')

In [42]:
for i in data['user']:
    user = int(i)
    """get user ID"""
    user_index = data[data.user == user].index.tolist()[0]
    """Get the tickers the user bought before"""
    known_user_likes = data_items.loc[user_index] #gets the row in matrix according to account ID
    known_user_likes = known_user_likes[known_user_likes >0].index.values #keep only the bought tickers
    
    """construct neighbourhood of tickers similar to ones already bought"""
    """data frame of top 6 most similar items to each bought item"""
    most_similar_to_likes = data_neighbours.loc[known_user_likes]
    #make list of all similar tickers into a list
    similar_list2 = most_similar_to_likes.values.tolist()
    """orders are not correct, need to append bought tickers to list"""
    """question: top 20 tickers instead?"""
    most_similar_to_likes_col = most_similar_to_likes.index.tolist()
    similar_list2.append(most_similar_to_likes_col)

    #make lists in list into a flat list, and dedupe 
    similar_list = []
    for sublist in similar_list2:
        for item in sublist:
            similar_list.append(item)
    similar_list = list(dict.fromkeys(similar_list))
    
    #make matrix of similarities between similar items
    neighbourhood = data_matrix2[similar_list].loc[similar_list]
    # user weights for each item neighbourhood and item bought before 
    user_vector = data_items.loc[user_index].loc[similar_list]
    """Calculate the score using formula"""
    """REEVALUATE SCORE METHOD"""
    #(dot product of neighbourhood and user rating)/sum of neighbourhood
    score2 = neighbourhood.dot(user_vector).div(neighbourhood.sum(axis=1))
    # Drop the tickers bought before 
    
    score = score2.drop(known_user_likes)
    top5 = score.nlargest(5)
    #top5df =pd.DataFrame({'tickers':top5.index, 'score':top5.values})
    data.iloc[user, data.columns.get_loc('bought tickers')] = str(known_user_likes)
    data.iloc[user, data.columns.get_loc('recommendation')] = str(top5.index.tolist())

In [43]:
df1 = data[['user','bought tickers', 'recommendation']]
df1.to_csv('finaldf.csv')